<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/MLP_TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
dataset=pd.read_csv("temperature.csv")
dataset.head()

In [ ]:
dataset.tail()

In [ ]:
df2 = dataset[["month", "day", "year"]].copy()
df_date=pd.to_datetime(df2)
df_date=pd.DataFrame(df_date)

In [ ]:
dfAT=dataset["AverageTemperatureFahr"]
df_date.insert(1,"AverageTemperatureFahr",dfAT )


In [ ]:
df_date

In [ ]:
df_date_renamed=df_date.rename(columns={0:"date"})

In [ ]:
df_date_renamed

In [ ]:
#df_date_renamed['date']=df_date_renamed['date'].apply(lambda x:x.value)

In [ ]:
df_nomissing=df_date_renamed.dropna()

In [ ]:
df_nomissing

In [ ]:
df_nomissing=df_nomissing.set_index('date')

In [ ]:
df_nomissing.columns

In [ ]:
df_nomissing

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

In [ ]:
#df_scaled=scaler.fit_transform(df_nomissing)

In [ ]:
#df_scaled

In [ ]:
plt.plot(df_nomissing)
plt.plot(figsize=(20,5))
plt.title("time series")
plt.show

In [ ]:
import statsmodels.tsa.stattools as sts

In [ ]:
sts.adfuller(df_nomissing.AverageTemperatureFahr)

5% estimate is -2.8616<br>
This is less than -2.8135<br>

The timeseries is stationary

In [ ]:
from numpy import array


In [ ]:
data_NM=np.array(df_nomissing)


In [ ]:
def series_to_supervised(data,n_in,n_out):

    df = pd.DataFrame(data)
    cols = list()
    for i in range(n_in,0,-1): cols.append(df.shift(i))
    for i in range(0, n_out): cols.append(df.shift(-i))
    agg = pd.concat(cols,axis=1)
    agg.dropna(inplace=True)

    return agg.values

In [ ]:
def to_supervised(train,n_input,n_out):
    #flaten data
    data=train
    X,y=list(),list()
    in_start=0
    for _ in range(len(data)):
        in_end=in_start+ n_input
        out_end=in_end + n_out
        if out_end<=len(data):
            x_input=data[ in_start:in_end,0]
            x_input=x_input.reshape((len(x_input)))
            X.append(x_input)
            y.append(data[in_end:out_end,0])
        in_start+=1
    return array(X), array(y)

In [ ]:
n_in,n_out = 2,1

In [ ]:
#df_supervised=series_to_supervised(df_nomissing, n_in,n_out)

In [ ]:
X,y=to_supervised(data_NM, 7,1)

Predict from 1 step and 7 days before

In [ ]:
#df_supervised[0]

In [ ]:
#df_super[0]

In [ ]:
#x, y= df_supervised[:,None,:-n_out*3],df_supervised[:,n_in*3:]
#X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=49)

In [ ]:
#x

In [ ]:
#y

In [ ]:
X_train0, X_test, y_train0, y_test = train_test_split(X, y, random_state=42)


In [ ]:
X_train1, X_validation, y_train1, y_validation = train_test_split(X_train0, y_train0, random_state=42)


In [ ]:
X_train1

In [ ]:
X_train_s

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))

In [ ]:
from keras.src.layers.serialization import preprocessing_normalization
from sklearn import preprocessing
import numpy as np



In [ ]:
X_train_s=scaler.fit_transform(X_train1)
X_validation_s=scaler.fit_transform(X_validation)
X_test_s=scaler.fit_transform(X_test)
y_train_s=scaler.fit_transform(y_train1)
y_validation_s=scaler.fit_transform(y_validation)
y_test_s=scaler.fit_transform(y_test)



In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Dropout

In [ ]:
model_mlp=Sequential()
model_mlp.add(Dense(10,activation="relu",input_dim=X_train_s.shape[1]))
model_mlp.add(Dense(20,activation="relu"))
model_mlp.add(Dense(30,activation="relu"))
model_mlp.add(Dense(n_step,activation="selu"))
model_mlp.compile(loss='mse',optimizer="adam")

In [ ]:
history_mlp=model_mlp.fit(X_train_s,y_train_s,epochs=100,
                          validation_data=(X_validation_s,y_validation_s)
              , batch_size=16)


In [ ]:
fig,ax=plt.subplots(figsize=(12,5))
ax.plot(history_mlp.history['loss'],label='train_loss')
ax.plot(history_mlp.history['val_loss'],label='validation_loss')
ax.legend()

In [ ]:
predict_mlp=model_mlp.predict(X_test_s)
predict_mlp.shape


In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
inv_y_test=scaler.inverse_transform(y_test_s)
inv_y_pred=scaler.inverse_transform(predict_mlp)


In [ ]:
fig=plt.figure(figsize=(20,10))
plt.plot(inv_y_test[:,:2],color='b',label="Real")
plt.plot(inv_y_pred,color='r',label="predicted")
plt.legend()
plt.show()

In [ ]:
X_test_s[-1]

In [ ]:
input_samples=X_test_s[-1:,:]
input_samples

In [ ]:
predict_mlp[-1]

In [ ]:
y_test_s[-1]

In [ ]:
next_step=model_mlp.predict(input_samples)
#next_step=scaler.inverse_transform(next_step)

In [ ]:
next_step